In [ ]:
import pandas as pd
import numpy as np
import warnings
import matplotlib.pyplot as plt
import rasterio
import geopandas as geopd
import rasterio.rio
import seaborn as sns
import datetime as dt 
import h3

from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split, GridSearchCV, RandomizedSearchCV
from sklearn.metrics import confusion_matrix, accuracy_score, f1_score, recall_score, classification_report, roc_auc_score, make_scorer

from xgboost import XGBClassifier


from rasterio.plot import show

import pyreadr

RSEED = 42

In [ ]:
df_all = geopd.read_file("../data/final_shapefiles/foxes_modelling_all.shp")
df_resamp = geopd.read_file("../data/final_shapefiles/foxes_modelling_resamp.shp")
df_sample = geopd.read_file("../data/final_shapefiles/sample_points.shp")

In [ ]:
#in a fist step, the category "N" is created twice

df_sample["asp"] = pd.cut(df_sample.aspect, 
                                bins = [-1.1,0,22.5,67.5,112.5,157.5,202.5,247.5,292.5,337.5,360],
                                labels = ["Flat", "N", "NE", "E", "SE", "S", "SW", "W", "NW", "N2"])
df_sample["asp"] = df_sample.asp.replace("N2","N")


In [ ]:
cat_variables = ["soil", "veg", "asp"]

In [ ]:
categories_sample = pd.get_dummies(df_sample[cat_variables], drop_first=True)

In [ ]:
df_sample = pd.concat([df_sample, categories_sample], axis = 1)

In [ ]:
df_all_enc = df_all.drop(["veg", "soil"], axis = 1)
df_resamp_enc = df_resamp.drop(["veg", "soil"], axis = 1)
df_sample_enc = df_sample.drop(["veg", "soil"], axis = 1)

## __Building Test and Train Set__

First we build a data frame with only the target and one with everything except the target.

In [ ]:
X = df_all_enc.drop("target", axis = 1)

y = df_all_enc["target"]

Then these data frames are split in two data frames for the test data set and the train data set.

In [ ]:
df_all_train, df_all_test, y_train, y_test = train_test_split(X, y, stratify = y, random_state = RSEED, test_size = 0.25)

Further, for the training, we have to drop all columns from our feature data set, that we cannot and/or do not want to use for our model. 

In [ ]:
df_all_train.columns

In [ ]:
X_train = df_all_train.iloc[:,5:].drop(["geometry", "area", "timestamp", "aspect"], axis = 1)
X_test = df_all_test.iloc[:,5:].drop(["geometry", "area", "timestamp", "aspect"], axis = 1)


## __XGBoost without Scaling of Positive Weight__

First we train a model without scaling of the positive weight, just do see, how it does on our data.

In [ ]:
xgb = XGBClassifier(random_state = RSEED)

xgb.fit(X_train, y_train)

In [ ]:
df_all_train["pred"] = xgb.predict(X_train)

In [ ]:
df_all_test["pred"] = xgb.predict(X_test)

In [ ]:
print(classification_report(y_train, df_all_train.pred))
print(classification_report(y_test, df_all_test.pred))

As we would like to optimize for recall, this does not have a satisfying outcome. The model is predicting over 20 % of the fox locations wrong. 

## __XGBoost with Scaling of Positive Weight__

To do something about that we scale up the positive weight, telling the model to put more weight on errors it does on the 1s in our target, so the known fox locations.

In [ ]:
xgb = XGBClassifier(scale_pos_weight = 100, random_state = RSEED)

In [ ]:
xgb.fit(X_train, y_train)

### __Prediction of the Training and Test Data Set__

We predict both the probability and the binary response for later use in the depiction of our results.

We start with the prediction of the training data.

In [ ]:
y_pred_train = xgb.predict_proba(X_train) # predict the probability of the outcome
y_pred_train_class = xgb.predict(X_train) # predict the actual class of the outcome, when the threshold is 0.5. This threshold can be changed. 

In [ ]:
y_pred_train = pd.DataFrame(y_pred_train) # as we want to join the probability prediction to the data frame, we convert the predictions to a data frame.

In [ ]:
df_all_train["pred_prob"] = np.array(y_pred_train.iloc[:,1:]) # we then join the prediction to our training data frame
df_all_train["pred"] = xgb.predict(X_train) # the class outcome can be directly predicted into a new column of the data frame

# we further build bins out of the probabilities that we can label as "suitable", "intermediate suitable" and "unsuitable" habitat for our foxes
df_all_train["proba_bin"] = pd.cut(df_all_train.pred_prob,
            bins = [0, 0.6, 0.8, 1],
            labels = [0, 0.8, 1]
).astype("float")

Now we do the same steps for the test data.

In [ ]:
y_pred_test = xgb.predict_proba(X_test) # predict the probability of the outcome
y_pred_test_class = xgb.predict(X_test) # predict the actual class of the outcome, when the threshold is 0.5. This threshold can be changed. 

In [ ]:
y_pred_test = pd.DataFrame(y_pred_test) # as we want to join the probability prediction to the data frame, we convert the predictions to a data frame.

In [ ]:
df_all_test["pred_prob"] = np.array(y_pred_test.iloc[:,1:]) # we then join the prediction to our test data frame
df_all_test["pred"] = xgb.predict(X_test) # the class outcome can be directly predicted into a new column of the data frame

# we further build bins out of the probabilities that we can label as "suitable", "intermediate suitable" and "unsuitable" habitat for our foxes
df_all_test["proba_bin"] = pd.cut(df_all_test.pred_prob,
            bins = [0, 0.6, 0.8, 1],
            labels = [0, 0.8, 1]
).astype("float")

Lastly we print the classification report again. As we can see, our recall is much higher. (In the end we might just want to show recall and maybe accuracy/ AUC, should talk about that). With a recall of 1 on our training and a recall of 0.98 on our test set, we are confident in our results.

In [ ]:
print(classification_report(y_train, y_pred_train_class))
print(classification_report(y_test, y_pred_test_class))

Now let us have a look into the importance of the different features for our XGBoost model.

In [ ]:
feat_importances = pd.DataFrame(xgb.feature_importances_, index=X_train.columns, columns=["Importance"])
feat_importances.sort_values(by='Importance', ascending=False, inplace=True)
feat_importances.plot(kind='bar', figsize=(8,6))

As we can see, elevation is the most important feature.

In [ ]:
print(roc_auc_score(y_train, y_pred_train_class))
print(roc_auc_score(y_test, y_pred_test_class))

And here is a confusion matrix for the test results as well.

In [ ]:
results = confusion_matrix(y_test, y_pred_test_class)
print(results)

In [ ]:
ax = sns.heatmap(results, annot = True, cmap = "Blues")
ax.set_title('XGBoost Confusion Matrix without Adjustments\n\n');
ax.set_xlabel('\nPredicted Values')
ax.set_ylabel('Actual Values ');

## Ticket labels - List must be in alphabetical order
ax.xaxis.set_ticklabels(['No Fox','Fox'])
ax.yaxis.set_ticklabels(['No Fox','Fox'])

## Display the visualization of the Confusion Matrix.
plt.show()

## __Predicting on regular Sample over whole Study Area__

As we are not allowed to show the locations of the foxes in the end, we now use our model to predict on a regular raster over the whole study area. First we have to drop some of the columns that are still in our sample set.

In [ ]:
sample_pred = df_sample_enc.drop(["x", "y", "geometry", "asp", "aspect"], axis = 1)

We have to rename some of our columns in the sample point data frame, as they do not match the names in the training data set.

In [ ]:
sample_pred.rename(columns = {"veg_Moist Shrub" : "veg_Moist",
                    "soil_Peat(Turf)" : "soil_Peat(",
                    "veg_Dry Shrub" : 'veg_Dry Sh',
                    "veg_Grassland" : "veg_Grassl",
                    "soil_Roesberg" : "soil_Roesb"}, inplace = True)

We then follow the same steps as above to predict the points.

In [ ]:
y_pred_sample = xgb.predict_proba(sample_pred)
y_pred_sample = pd.DataFrame(y_pred_sample)
y_pred_sample_1 = np.array(y_pred_sample.iloc[:,1:])
df_sample_enc["pred_prob"] = y_pred_sample_1
df_sample_enc["pred"] = xgb.predict(sample_pred)
df_sample_enc["proba_bin"] = pd.cut(df_sample_enc.pred_prob,
            bins = [0, 0.6, 0.8, 1],
            labels = [0, 0.8, 1]
).astype("float")

# __Grid Search for XGBoost__

We do not really do that. But we could search for different parameters in a grid search. But for now you can more or less ignore this section.

In [ ]:
def my_custom_score(y_true, y_pred):
    return f1_score(y_true, y_pred) + recall_score(y_true, y_pred)

my_scorer = make_scorer(my_custom_score)

In [ ]:
#param_grid = {"scale_pos_weight" : [1, 20, 40, 60, 80, 100]}
param_grid = {"scale_pos_weight" : [3, 5, 8]}

In [ ]:
xgb = XGBClassifier()

In [ ]:
def multi_scoring(clf, X, y):
    y_pred = clf.predict(X)
    f1_pred = f1_score(y, y_pred)
    recall = recall_score(y, y_pred)
    score_sum = f1_pred + recall
    return {"f1": f1_pred, "recall": recall, "score_sum": score_sum}

In [ ]:
# grid = GridSearchCV(estimator = xgb, param_grid = param_grid, verbose = 5, cv = 5, n_jobs = -1, scoring = ["recall", "f1_macro"], refit = "recall" + "f1_macro")
grid = GridSearchCV(estimator = xgb, param_grid = param_grid, verbose = 5, cv = 5, n_jobs = -1, scoring = multi_scoring, refit = "score_sum")

In [ ]:
grid_result = grid.fit(X_train, y_train)

In [ ]:
grid_result.best_params_

In [ ]:
#param_grid = {"scale_pos_weight" : [1, 20, 40, 60, 80, 100]}
param_grid = {"scale_pos_weight" : [3, 5, 8]}

In [ ]:
# grid = GridSearchCV(estimator = xgb, param_grid = param_grid, verbose = 5, cv = 5, n_jobs = -1, scoring = ["recall", "f1_macro"], refit = "recall" + "f1_macro")
grid = GridSearchCV(estimator = xgb, param_grid = param_grid, verbose = 5, cv = 5, n_jobs = -1, scoring = multi_scoring, refit = "score_sum")

In [ ]:
grid_result = grid.fit(X_train, y_train)

In [ ]:
grid_result.best_params_

In [ ]:
#param_grid = {"scale_pos_weight" : [1, 20, 40, 60, 80, 100]}
param_grid = {"scale_pos_weight" : [4, 5, 7]}

In [ ]:
# grid = GridSearchCV(estimator = xgb, param_grid = param_grid, verbose = 5, cv = 5, n_jobs = -1, scoring = ["recall", "f1_macro"], refit = "recall" + "f1_macro")
grid = GridSearchCV(estimator = xgb, param_grid = param_grid, verbose = 5, cv = 5, n_jobs = -1, scoring = multi_scoring, refit = "score_sum")

In [ ]:
grid_result = grid.fit(X_train, y_train)

In [ ]:
grid_result.best_params_

In [ ]:
#param_grid = {"scale_pos_weight" : [1, 20, 40, 60, 80, 100]}
param_grid = {"scale_pos_weight" : [5, 6]}

In [ ]:
# grid = GridSearchCV(estimator = xgb, param_grid = param_grid, verbose = 5, cv = 5, n_jobs = -1, scoring = ["recall", "f1_macro"], refit = "recall" + "f1_macro")
grid = GridSearchCV(estimator = xgb, param_grid = param_grid, verbose = 5, cv = 5, n_jobs = -1, scoring = multi_scoring, refit = "score_sum")

In [ ]:
grid_result = grid.fit(X_train, y_train)

In [ ]:
grid_result.best_params_

## __Trying to use H3 to convert Points to Hexagrid__

For now just ignore this section and move one to the next

In [ ]:
df_all_train.set_crs(3006, inplace = True)
df_all_train.to_crs(4326, inplace = True)

In [ ]:
df_all_train["lon"]  = df_all_train.geometry.x
df_all_train["lat"]  = df_all_train.geometry.y
df_all_train.head()

In [ ]:
def plot_scatter(df, metric_col, x='lon', y='lat', marker='.', alpha=1, figsize=(16,12), colormap='viridis'):    
    df.plot.scatter(x=x, y=y, c=metric_col, title=metric_col
                    , edgecolors='none', colormap=colormap, marker=marker, alpha=alpha, figsize=figsize);
    plt.xticks([], []); plt.yticks([], [])

In [ ]:
APERTURE_SIZE = 8
hex_col = 'hex'+str(APERTURE_SIZE)

# find hexs containing the points
df_all_train[hex_col] = df_all_train.apply(lambda x: h3.geo_to_h3(x.lon,x.lat,APERTURE_SIZE),1)



In [ ]:
sns.histplot(data = df_all_train, x = "proba_bin")

In [ ]:
# aggregate the points
# df_all_train_g = df_all_train.groupby(hex_col).size().to_frame('pred').reset_index()
df_all_train_g = df_all_train.groupby(hex_col).max().reset_index()

#find center of hex for visualization
df_all_train_g['lon'] = df_all_train_g[hex_col].apply(lambda x: h3.h3_to_geo(x)[0])
df_all_train_g['lat'] = df_all_train_g[hex_col].apply(lambda x: h3.h3_to_geo(x)[1])

# pltot the hexs
plot_scatter(df_all_train_g, metric_col='pred_prob', marker='o',figsize=(20,18))
plt.title('hex-grid: foxes');

In [ ]:
df_all_train_g

In [ ]:
df_all_train.head()

## __Plot of Prediction on whole area__

Here we plot the prediction we did on the whole area set. We first build a function to plot our outcome later.

In [ ]:
def plot_scatter(df, metric_col, x='lon', y='lat', marker='.', alpha=1, figsize=(16,12), colormap='viridis'):    
    df.plot.scatter(x=x, y=y, c=metric_col, title=metric_col
                    , edgecolors='none', colormap=colormap, marker=marker, alpha=alpha, figsize=figsize);
    plt.xticks([], []); plt.yticks([], [])

Then we set and change the crs of our data set.

In [ ]:
df_sample_enc.set_crs(3006, inplace = True)
df_sample_enc.to_crs(4326, inplace = True)

Next we extract the longitude and latitude from our geometry.

In [ ]:
df_sample_enc["lon"]  = df_sample_enc.geometry.x
df_sample_enc["lat"]  = df_sample_enc.geometry.y

Now we start using h3. With h3 we aggregate all of our points into hexagons of different sizes. We will use size 8, 9 or 10. I will plot examples for all 3 here. 

In [ ]:
APERTURE_SIZE = 8
hex_col = 'hex'+str(APERTURE_SIZE)

# find hexs containing the points
df_sample_enc[hex_col] = df_sample_enc.apply(lambda x: h3.geo_to_h3(x.lon,x.lat,APERTURE_SIZE),1)



In [ ]:
# aggregate the points
df_sample_enc_g8 = df_sample_enc.groupby(hex_col).mean().reset_index()

#find center of hex for visualization
df_sample_enc_g8['lon'] = df_sample_enc_g8[hex_col].apply(lambda x: h3.h3_to_geo(x)[0])
df_sample_enc_g8['lat'] = df_sample_enc_g8[hex_col].apply(lambda x: h3.h3_to_geo(x)[1])


In [ ]:
# pltot the hexs
plot_scatter(df_sample_enc_g8, metric_col='pred_prob',figsize=(11,9), marker='h')
plt.title('hex-grid: foxes');

In [ ]:
APERTURE_SIZE = 9
hex_col = 'hex'+str(APERTURE_SIZE)

# find hexs containing the points
df_sample_enc[hex_col] = df_sample_enc.apply(lambda x: h3.geo_to_h3(x.lon,x.lat,APERTURE_SIZE),1)



In [ ]:
# aggregate the points
df_sample_enc_g9 = df_sample_enc.groupby(hex_col).mean().reset_index()

#find center of hex for visualization
df_sample_enc_g9['lon'] = df_sample_enc_g9[hex_col].apply(lambda x: h3.h3_to_geo(x)[0])
df_sample_enc_g9['lat'] = df_sample_enc_g9[hex_col].apply(lambda x: h3.h3_to_geo(x)[1])


In [ ]:
# pltot the hexs
plot_scatter(df_sample_enc_g9, metric_col='pred_prob',figsize=(22,20), marker='h')
plt.title('hex-grid: foxes');

In [ ]:
APERTURE_SIZE = 10
hex_col = 'hex'+str(APERTURE_SIZE)

# find hexs containing the points
df_sample_enc[hex_col] = df_sample_enc.apply(lambda x: h3.geo_to_h3(x.lon,x.lat,APERTURE_SIZE),1)



In [ ]:
# aggregate the points
df_sample_enc_g = df_sample_enc.groupby(hex_col).mean().reset_index()

#find center of hex for visualization
df_sample_enc_g['lon'] = df_sample_enc_g[hex_col].apply(lambda x: h3.h3_to_geo(x)[0])
df_sample_enc_g['lat'] = df_sample_enc_g[hex_col].apply(lambda x: h3.h3_to_geo(x)[1])


In [ ]:
# pltot the hexs
plot_scatter(df_sample_enc_g, metric_col='pred_prob',figsize=(40,38), marker='h')
plt.title('hex-grid: foxes');

All the ones above are plotted with the mean. The one below is plotted with the max.

In [ ]:
# aggregate the points
# df_sample_enc_g = df_sample_enc.groupby("hex8").size().to_frame('pred').reset_index()
df_sample_enc_g = df_sample_enc.groupby("hex8").max().reset_index()

#find center of hex for visualization
df_sample_enc_g['lon'] = df_sample_enc_g["hex8"].apply(lambda x: h3.h3_to_geo(x)[0])
df_sample_enc_g['lat'] = df_sample_enc_g["hex8"].apply(lambda x: h3.h3_to_geo(x)[1])

# pltot the hexs
plot_scatter(df_sample_enc_g, metric_col='pred_prob', marker='h',figsize=(11,9))
plt.title('hex-grid: foxes');

In [ ]:
# aggregate the points
# df_sample_enc_g = df_sample_enc.groupby("hex8").size().to_frame('pred').reset_index()
df_sample_enc_g = df_sample_enc.groupby("hex8").count().reset_index()
df_sample_enc_g.head()